In [81]:
### Import Library

import pandas as pd
import numpy as np
import random
import operator
import math
from google.colab import files
uploaded = files.upload()

Saving Data.csv to Data (6).csv


In [82]:
### reading the input csv file

df_full = pd.read_csv("Data.csv")
columns = list(df_full.columns)
features = columns[:len(columns)-1]
class_labels = list(df_full[columns[-1]])
df = df_full[features]

In [83]:
# Jumlah atribut
num_attr = len(df.columns) - 1

# Jumlah Cluster
k = 2

# Maksimum Iterasi
MAX_ITER = 100

# jumlah data poin
n = len(df)

# Fuzzy parameter
m = 2.00

In [84]:
# Fungsi akurasi untuk menghitung akurasi dari hasil

def accuracy(cluster_labels, class_labels):
  county = [0,0]
  countn = [0,0]
  tp = [0, 0]
  tn = [0, 0]
  fp = [0, 0]
  fn = [0, 0]

  for i in range(len(df)):
    # Yes = 1, No = 0
    if cluster_labels[i] == 1 and class_labels[i] == 'Yes':
      tp[0] = tp[0] + 1
    if cluster_labels[i] == 0 and class_labels[i] == 'No':
      tn[0] = tn[0] + 1
    if cluster_labels[i] == 1 and class_labels[i] == 'No':
      fp[0] = fp[0] + 1
    if cluster_labels[i] == 0 and class_labels[i] == 'Yes':
      fn[0] = fn[0] + 1

  for i in range(len(df)):
    # Yes = 0, No = 1
    if cluster_labels[i] == 0 and class_labels[i] == 'Yes':
      tp[1] = tp[1] + 1
    if cluster_labels[i] == 1 and class_labels[i] == 'No':
      tn[1] = tn[1] + 1
    if cluster_labels[i] == 0 and class_labels[i] == 'No':
      fp[1] = fp[1] + 1
    if cluster_labels[i] == 1 and class_labels[i] == 'Yes':
      fn[1] = fn[1] + 1

  a0 = float((tp[0] + tn[0]))/(tp[0] + tn[0] + fn[0] + fp[0])
  a1 = float((tp[1] + tn[1]))/(tp[1] + tn[1] + fn[1] + fp[1])
  p0 = float(tp[0])/(tp[0] + fp[0])
  p1 = float(tp[1])/(tp[1] + fp[1])
  r0 = float(tp[0])/(tp[0] + fn[0])
  r1 = float(tp[1])/(tp[1] + fn[1])

  accuracy = [a0*100,a1*100]
  precision = [p0*100,p1*100]
  recall = [r0,100,r1*100]

  return accuracy,precision, recall

In [85]:
### Inisialisasi Jumlah matriks dengan random value

n = len(df)
k = 2
def initializeMembershipMatrix(n, k):
  membership_mat = list()
  for i in range(n):
    random_num_list = [random.random() for i in range(k)]
    summation = sum(random_num_list)
    temp_list = [x/summation for x in random_num_list]
    membership_mat.append(temp_list)
  return membership_mat

In [86]:
### Menghitung pusat cluster dengan setiap iterasi

def calculateClusterCenter(membership_mat):
  cluster_mem_val = list(zip(*membership_mat))
  cluster_centers = list()
  for j in range(k):
    x = list(cluster_mem_val[j])
    xraised = [e ** m for e in x]
    denominator = sum(xraised)
    temp_num = list()
    for i in range(n):
      data_point = list(df.iloc[i])
      prod = [xraised[i] * val for val in data_point]
      temp_num.append(prod)
    numerator = map(sum, zip(*temp_num))
    center = [z/denominator for z in numerator]
    cluster_centers.append(center)
  return cluster_centers

In [87]:
### update nilai dengan pusat cluster
def updateMembershipValue(membership_mat, cluster_centers):
  p = float(2/(m-1))
  for i in range(n):
    x = list(df.iloc[i])
    distances = [np.linalg.norm(list(map(operator.sub, x, cluster_centers[j]))) for j in range (k)]
    for j in range(k):
      den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
      membership_mat[i][j] = float(1/den)
  return membership_mat

In [88]:
def getClusters(membership_mat):
  cluster_labels = list()
  for i in range(n):
    max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
    cluster_labels.append(idx)
  return cluster_labels

In [89]:
### final fungsi FCM

def fuzzyCMeansClustering():
  n = len(df)
  k = 2

  # Membership Matrix
  membership_mat = initializeMembershipMatrix(n, k)
  curr = 0
  while curr <= MAX_ITER:
      cluster_centers = calculateClusterCenter(membership_mat)
      membership_mat = updateMembershipValue(membership_mat, cluster_centers)
      cluster_labels = getClusters(membership_mat)
      curr += 1

  print(membership_mat)
  return cluster_labels, cluster_centers


In [90]:
### mengambil fungsi utama dan mengirimkan hasil akhir dalam label dan pusat cluster

print("printing the final membership matrix")

labels, centers = fuzzyCMeansClustering()

print("printing cluster centers")
print(centers)

### menghitug hasil akurasi

a,p,r = accuracy(labels, class_labels)

### cetak nilai akurasi

print("Accuracy = " + str(a))
print("Precision = " + str(p))
print("Recall = " + str(r))

printing the final membership matrix
[[0.49716035198557545, 0.5028396480144245], [0.5016143574795722, 0.49838564252042783], [0.5002573994939595, 0.4997426005060406], [0.5018063955848198, 0.4981936044151803], [0.49868833409191193, 0.5013116659080881], [0.496428326570517, 0.5035716734294831], [0.49791608220549727, 0.5020839177945028], [0.49723207607757725, 0.5027679239224228], [0.4990853609102951, 0.5009146390897049], [0.496234169706508, 0.503765830293492], [0.4979877032974125, 0.5020122967025874], [0.49823946152032256, 0.5017605384796775], [0.5006138300482129, 0.4993861699517872], [0.4973566257936892, 0.5026433742063108], [0.4970737028999701, 0.5029262971000299], [0.4983917385326349, 0.501608261467365], [0.49595420030618304, 0.5040457996938169], [0.4989555843937893, 0.5010444156062108], [0.4992997644206592, 0.5007002355793407], [0.5028525075261007, 0.49714749247389933], [0.4969495591400028, 0.5030504408599972], [0.4966701791538847, 0.5033298208461152], [0.5026421984823781, 0.49735780151